In [1]:
import os
import numpy as np
import trimesh
from scipy.ndimage import zoom
from tqdm import tqdm
import kagglehub

In [2]:
# Download latest version
path = kagglehub.dataset_download("balraj98/modelnet40-princeton-3d-object-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\amirh\.cache\kagglehub\datasets\balraj98\modelnet40-princeton-3d-object-dataset\versions\1


In [5]:
dataset_root = os.path.join(path, "ModelNet40")  
selected_classes = [
    'airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car',
    'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot',
    'glass_box', 'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor',
    'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink',
    'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase',
    'wardrobe', 'xbox'
]
voxel_dim = 32 

In [7]:
def mesh_to_voxel(path_to_off_file, voxel_dim=32):
    mesh = trimesh.load(path_to_off_file)
    voxelized = mesh.voxelized(pitch=mesh.scale / voxel_dim)
    voxel_matrix = voxelized.matrix.astype(np.float32)

    voxel_matrix = zoom(voxel_matrix, (
        voxel_dim / voxel_matrix.shape[0],
        voxel_dim / voxel_matrix.shape[1],
        voxel_dim / voxel_matrix.shape[2]
    ))

    return voxel_matrix

In [9]:
def load_voxel_dataset(dataset_path, voxel_dim=32, selected_classes=None):
    X, y = [], []
    if selected_classes is None:
        selected_classes = sorted(os.listdir(dataset_path))
    
    label_map = {cls_name: idx for idx, cls_name in enumerate(selected_classes)}

    for cls_name in tqdm(selected_classes, desc="Classes"):
        class_path = os.path.join(dataset_path, cls_name, "train")
        if not os.path.exists(class_path):
            print(f"❌ : {class_path}")
            continue
        
        file_list = [f for f in os.listdir(class_path) if f.endswith('.off')]
        for file_name in tqdm(file_list, desc=f"{cls_name}", leave=False):
            file_path = os.path.join(class_path, file_name)
            try:
                voxel = mesh_to_voxel(file_path, voxel_dim)
                X.append(voxel)
                y.append(label_map[cls_name])
            except Exception as e:
                print(f"⚠️{file_path}: {e}")
    
    X = np.array(X)[..., np.newaxis]
    y = np.array(y)
    return X, y, label_map

In [ ]:
X, y, label_map = load_voxel_dataset(dataset_root, voxel_dim, selected_classes)

# 💾 
np.save("MN40_X_voxel_full.npy", X)
np.save("MN40_y_voxel_full.npy", y)


print("✅")
print("🔢:", X.shape, y.shape)
print("🏷️:", label_map)

curtain:  47%|████▋     | 65/138 [02:37<03:40,  3.01s/it]C:\Users\amirh\anaconda3\Lib\site-packages\trimesh\grouping.py:99: RuntimeWarning: invalid value encountered in cast
  stacked = np.column_stack(stacked).round().astype(np.int64)

vase:   3%|▎         | 16/475 [00:05<04:05,  1.87it/s]


chair:  31%|████████████████████████████████████████▍                                                                                        | 279/889 [00:59<01:35,  6.39it/s]


chair:  38%|████████████████████████████████████████████████▊                                                                                | 336/889 [01:13<01:28,  6.24it/s]


chair:  45%|█████████████████████████████████████████████████████████▌                                                                       | 397/889 [01:25<05:13,  1.57it/s]


chair:  51%|█████████████████████████████████████████████████████████████████▉                                                               | 454/889 [01:41<01:20,  5.40it/s]


...

dresser:  46%|███████████████████████████████████████████████████████████▌                                                                    | 93/200 [00:29<00:46,  2.32it/s]


dresser:  74%|█████████████████████████████████████████████████████████████████████████████████████████████▎                                 | 147/200 [00:57<00:29,  1.79it/s]


dresser:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 197/200 [01:19<00:00,  3.74it/s]


Classes: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [19:50<00:00, 119.06s/it]


✅ آماده‌سازی دیتا تمام شد.
🔢 شکل داده‌ها: (3991, 32, 32, 32, 1) (3991,)
🏷️ لیبل‌ها: {'chair': 0, 'table': 1, 'sofa': 2, 'monitor': 3, 'desk': 4, 'bed': 5, 'bathtub': 6, 'night_stand': 7, 'toilet': 8, 'dresser': 9}


In [16]:
num_classes = len(np.unique(y))  # len(label_map)
print(num_classes)

10
